# Exploring the Veeam API with Python

The aim of this notebook is to provide you with how you can work with the Veeam API using Python.

There are some tricky parts to using the API initally, however, once you get past this the API is relatively easy to consume. 

You are viewing a Jupyter Notebook, here you are able execute the code in each of the cells, this includes entering in information to run certain commands. 

You will need to press 'SHIFT + ENTER' to execute the code in each cell.

## Creating the session

The first step in the use of the API is creating a session, this is done by making a GET request to:

    https://enterprise_manager:9893/api/
    
With Python we use the Requests library to handle all API requests to the server. First you need to import the library after installing, if you don't have it already do the following:

    pip install requests

Of course you'll also need Python from https://www.python.org/ remember to include python in your environmental path.

First step is to import the libraries we will need to explore the API.

In [181]:
import requests
import urllib3
import pprint

Next we need to make the request to the API, replace the URL with the IP/DNS of your Enterprise Manager.

Running this sends back quite a complex json object which was daughting to me when I first saw it.

In [182]:
HOST = "192.168.3.10"
PORT = "9398"
urllib3.disable_warnings()

url = f"https://{HOST}:{PORT}/api/"
headers = {"Accept": "application/json"}
verify = False

res = requests.get(url, headers=headers, verify=verify)
print(f" Response code: {res.status_code}")
res_json = res.json()
pprint.pprint(res.json())

 Response code: 200
{'Href': None,
 'Links': [{'Href': 'https://192.168.3.10:9398/api/logonSessions',
            'Name': None,
            'Rel': 'Down',
            'Type': 'LogonSessionList'},
           {'Href': 'https://192.168.3.10:9398/api/sessionMngr/?v=latest',
            'Name': None,
            'Rel': 'Create',
            'Type': 'LogonSession'}],
 'SupportedVersions': {'SupportedVersions': [{'Links': [{'Href': 'https://192.168.3.10:9398/api/sessionMngr/?v=v1',
                                                         'Name': None,
                                                         'Rel': 'Create',
                                                         'Type': 'LogonSession'}],
                                              'Name': 'v1'},
                                             {'Links': [{'Href': 'https://192.168.3.10:9398/api/sessionMngr/?v=v1_1',
                                                         'Name': None,
                                          

We need to use one of the links listed above, usually the latest version, to perform login.

You can either copy the URL or you can grab it using code.

If you want to do it by code do the following, though it isn't exactly easy!

In [183]:
request_url = res_json['SupportedVersions']['SupportedVersions'][-1]['Links'][0]['Href']
print(request_url)

https://192.168.3.10:9398/api/sessionMngr/?v=v1_5


## Logging in

Next we need to log in to the API using our credentails to get a X-RestSvcSessionId (token) back from the API to allow us to make our API requests.

Getting the token is one of the harder aspects of the API, fortunatly Python has some methods to make it abit easier. 

So the main issue in regards to the token is that it is sent back in a lump of code, a bit like the example above.

To send the request to the server for the token we do the following:

In [184]:
import getpass
login_url = f"https://{HOST}:{PORT}/api/sessionMngr/?v=1_5"
username = input('Enter Username: ')
password = getpass.getpass("Enter password: ")
    
response = requests.post(login_url, auth=requests.auth.HTTPBasicAuth(username, password), verify=verify)

print(response.status_code)
res_headers = response.headers
pprint.pprint(dict(res_headers))

Enter Username:  administrator@testlab.net
Enter password:  ········


201
{'Content-Length': '5978',
 'Content-Type': 'application/xml; charset=utf-8',
 'Date': 'Fri, 15 Jan 2021 20:28:26 GMT',
 'Location': 'https://192.168.3.10:9398/api/sessionMngr/7a3980e0-5c52-4678-9c1c-b77c54edb078',
 'Server': 'Microsoft-HTTPAPI/2.0',
 'Set-Cookie': 'X-RestSvcSessionId=N2EzOTgwZTAtNWM1Mi00Njc4LTljMWMtYjc3YzU0ZWRiMDc4; '
               'path=/api',
 'X-RestSvcSessionId': 'N2EzOTgwZTAtNWM1Mi00Njc4LTljMWMtYjc3YzU0ZWRiMDc4'}


Note that getpass is a library that allows us to securely input the password.

So you should be seeing the headers that have been send back from the server above which should be an object. we need to grab the 'X-RestSvcSessionId' key-value pair out and then add it to the current headers object.

To address this we will use a python 'get' method, this grabs a value from a object above.

In [185]:
token = res_headers.get('X-RestSvcSessionId')
print(token)

N2EzOTgwZTAtNWM1Mi00Njc4LTljMWMtYjc3YzU0ZWRiMDc4


The add that back to the original header

In [186]:
headers['X-RestSvcSessionId'] = token
print(headers)

{'Accept': 'application/json', 'X-RestSvcSessionId': 'N2EzOTgwZTAtNWM1Mi00Njc4LTljMWMtYjc3YzU0ZWRiMDc4'}


## Test a request

Right, that is basically it, we can now start making requests to the API. 

For this test we'll be using the /backups request to get a list of the backups.

In [187]:
base_url = f"https://{HOST}:{PORT}/api"

bu_addon = base_url + "/backups"

response = requests.get(bu_addon, headers=headers, verify=verify)

response_json = response.json()
pprint.pprint(response_json)

{'Refs': [{'Href': 'https://192.168.3.10:9398/api/backups/3b0627ee-84b7-4eb7-979d-44e5243273f5',
           'Links': [{'Href': 'https://192.168.3.10:9398/api/backupServers/08d7c6c0-39c6-4822-828a-e6ea1ca9b192',
                      'Name': 'veeamv10.testlab.net',
                      'Rel': 'Up',
                      'Type': 'BackupServerReference'},
                     {'Href': 'https://192.168.3.10:9398/api/repositories/9ded35e2-9b74-4b92-9205-e0fb5ad8d699',
                      'Name': 'v10_server_disk1',
                      'Rel': 'Up',
                      'Type': 'RepositoryReference'},
                     {'Href': 'https://192.168.3.10:9398/api/backups/3b0627ee-84b7-4eb7-979d-44e5243273f5?format=Entity',
                      'Name': 'AD Backup',
                      'Rel': 'Alternate',
                      'Type': 'Backup'},
                     {'Href': 'https://192.168.3.10:9398/api/backups/3b0627ee-84b7-4eb7-979d-44e5243273f5/restorePoints',
                      

We can get the names of each of the jobs via:

In [188]:
names = [item['Name'] for item in response_json['Refs']]
print(names)

['AD Backup', 'SQL1', 'solaris main']


## Jobs

Now we're going to look at the jobs

In [189]:
job_url = base_url + "/jobs"

response = requests.get(job_url, headers=headers, verify=verify)

response_json = response.json()
pprint.pprint(response_json)

{'Refs': [{'Href': 'https://192.168.3.10:9398/api/jobs/4dedb517-d61e-4fbe-88ce-7a396ab7839b',
           'Links': [{'Href': 'https://192.168.3.10:9398/api/backupServers/08d7c6c0-39c6-4822-828a-e6ea1ca9b192',
                      'Name': 'veeamv10.testlab.net',
                      'Rel': 'Up',
                      'Type': 'BackupServerReference'},
                     {'Href': 'https://192.168.3.10:9398/api/jobs/4dedb517-d61e-4fbe-88ce-7a396ab7839b?format=Entity',
                      'Name': 'AD Backup',
                      'Rel': 'Alternate',
                      'Type': 'Job'},
                     {'Href': 'https://192.168.3.10:9398/api/jobs/4dedb517-d61e-4fbe-88ce-7a396ab7839b/backupSessions',
                      'Name': None,
                      'Rel': 'Down',
                      'Type': 'BackupJobSessionReferenceList'}],
           'Name': 'AD Backup',
           'Type': 'JobReference',
           'UID': 'urn:veeam:Job:4dedb517-d61e-4fbe-88ce-7a396ab7839b'}]}


In [190]:
job_names = [item['Name'] for item in response_json['Refs']]
print(job_names)

['AD Backup']


## Overview Report

Next we're going to run the overview report

In [191]:
overview = base_url + "/reports/summary/overview"

overview_res = requests.get(overview, headers=headers, verify=verify)

overview_json = overview_res.json()

for key, value in overview_json.items():
    print(key, ' : ', value)


BackupServers  :  1
ProxyServers  :  2
RepositoryServers  :  6
RunningJobs  :  0
ScheduledJobs  :  1
SuccessfulVmLastestStates  :  1
WarningVmLastestStates  :  0
FailedVmLastestStates  :  0
Links  :  None
Href  :  None
Type  :  None


## Make a HTML Report

For this I'm going to use a package called 'dominate':

    pip install dominate
    
It makes it relatively easy to put together a nice looking report with styling that I'm importing from a Bootstrap CDN (Bootstrap is a popular CSS library).

In [192]:
import dominate
from dominate.tags import *

In [193]:
table_headers = ['Area', 'State']

def create_page():
    doc = dominate.document(title='Overview Report')        
    
    with doc.head:
        link(href="https://cdn.jsdelivr.net/npm/bootstrap@5.0.0-beta1/dist/css/bootstrap.min.css", rel="stylesheet", integrity="sha384-giJF6kkoqNQ00vy+HMDP7azOuL0xtbfIcaT9wjKHr8RbDVddVHyTfAAsrekwKmP1", crossorigin="anonymous")
        script(src="https://cdn.jsdelivr.net/npm/bootstrap@5.0.0-beta1/dist/js/bootstrap.bundle.min.js", integrity="sha384-ygbV9kiqUc6oa4msXn9868pTtWMgiQaeYH7/t7LECLbyPA2x65Kgf80OJFdroafW", crossorigin="anonymous")
    with doc:
        with div(cls='container'):
            h1('Overview Report')
            with table(cls='table table-striped'):
                with thead():
                    with tr():
                        for table_head in table_headers:
                            th(table_head)
                with tbody():
                    for key, value in overview_json.items():
                        with tr():
                            td(key)
                            td(str(value))
    
    with open('index.html', 'w') as file:
        file.write(doc.render())                  
create_page()

import os
os.startfile(r'.\index.html') 

## Job statistics

Next up job statistics

In [156]:
job_stats_url = base_url + "/reports/summary/job_statistics"

job_stats_res = requests.get(job_stats_url, headers=headers, verify=verify)

job_stats_json = job_stats_res.json()

In [180]:
job_stats_json

{'RunningJobs': 0,
 'ScheduledJobs': 1,
 'ScheduledBackupJobs': 1,
 'ScheduledReplicaJobs': 0,
 'TotalJobRuns': 1,
 'SuccessfulJobRuns': 1,
 'WarningsJobRuns': 0,
 'FailedJobRuns': 0,
 'MaxJobDuration': 240,
 'MaxBackupJobDuration': 240,
 'MaxReplicaJobDuration': 0,
 'MaxDurationBackupJobName': 'AD Backup',
 'MaxDurationReplicaJobName': '',
 'BackupJobStatusReportLink': 'Workspace/ViewReport.aspx?definition=7962844d-db6c-4d29-8b6e-4e0f7db0785f&ShowParams=1',
 'Links': None,
 'Href': None,
 'Type': None}

## NAS Jobs

Aim to pull out the files and folders

In [158]:
nas_jobs_url = base_url + "/nas/jobs"

nas_jobs_res = requests.get(nas_jobs_url, headers=headers, verify=verify)

nas_jobs_json = nas_jobs_res.json()

In [168]:
nas_uid = nas_jobs_json['Refs'][0]['UID']

In [170]:
nas_files_url = f"{nas_jobs_url}/{nas_uid}/includes"

nas_files_res = requests.get(nas_files_url, headers=headers, verify=verify)

nas_files_json = nas_files_res.json()

pprint.pprint(nas_files_json)

{'NASObjects': [{'ExclusionMask': {'Extensions': ['\\\\192.168.2.15\\Guitar\\.snapshot',
                                                  '\\\\192.168.2.15\\Guitar\\~snapshot']},
                 'FileOrFolder': '\\\\192.168.2.15\\Guitar',
                 'FileServerUid': 'urn:veeam:FileServer:f00ba514-0e5d-4911-b70f-6b33b0dc5d09',
                 'HierarchyObjRef': 'urn:NasBackup:BackupServer:ece1400a-5d35-4534-b175-ba3a97f2ce4e',
                 'Href': 'https://192.168.3.10:9398/api/nas/jobs/dad7f455-ee03-408c-a5cd-23c1f58f8845/includes/fc2c0999-08c1-4829-bb16-c78f4899bf14',
                 'InclusionMask': {'Extensions': ['*.*']},
                 'Links': [{'Href': 'https://192.168.3.10:9398/api/nas/jobs/dad7f455-ee03-408c-a5cd-23c1f58f8845?format=Entity',
                            'Name': 'File Backup Job 1',
                            'Rel': 'Up',
                            'Type': 'Job'},
                           {'Href': 'https://192.168.3.10:9398/api/nas/fileServer

## Conculsion

In this document we have explorered the Veeam API showing mainly of the more difficult aspects of creating the session, logging in, grabbing the token and then using it to make our requests.

I hope you have enjoyed!

Ed Howard